In [ ]:
!pip install supabase tf2onnx prophet joblib

In [ ]:
from keras.models import Sequential
from keras.layers import SimpleRNN, LSTM, GRU, Dense
from sklearn.preprocessing import MinMaxScaler
import pickle
from supabase import create_client, Client
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone
import plotly.express as px
import tensorflow as tf
import tf2onnx
from prophet import Prophet
import joblib

In [ ]:
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

In [ ]:
# FUNCION DE DESCARGA DE DATOS 
def get_demanda_data(page_size=1000):
    end = datetime.now(timezone.utc)
    start = end - timedelta(days=3650)
    all_data = []
    offset = 0
    while True:
        response = (
            supabase.table("demanda")
            .select("datetime,value")
            .gte("datetime", start.isoformat())
            .lte("datetime", end.isoformat())
            .range(offset, offset + page_size - 1)
            .execute()
        )
        data = response.data
        if not data:
            break
        all_data.extend(data)
        offset += page_size
        if len(data) < page_size:
            break

    if not all_data:
        return pd.DataFrame()
    df = pd.DataFrame(all_data)
    df["datetime"] = pd.to_datetime(df["datetime"])
    return df

# FUNCION DE ESCALADO Y GUARDADO DEL SCALER
def escalar_datos(df, save_path='scaler.pkl'):
    scaler = MinMaxScaler()
    df['value_scaled'] = scaler.fit_transform(df[['value']])

    with open(save_path, 'wb') as f:
        pickle.dump(scaler, f)

    print(f" Datos escalados y scaler guardado en '{save_path}'.")
    return df, scaler

# FUNCION PARA DIVIDIR EN TRAIN Y TEST
def dividir_train_test(df, test_size=0.2):
    df = df.copy()
    df = df.set_index('datetime')
    split_point = int(len(df) * (1 - test_size))
    df_train = df.iloc[:split_point]
    df_test = df.iloc[split_point:]
    return df_train, df_test


In [ ]:
# Cargar datos
df = get_demanda_data()
df, scaler = escalar_datos(df)
# Guardar dataset como CSV para usarlo en Streamlit
df.to_csv('datos_prediccion.csv', index=False)
print("✅ Dataset guardado como 'datos_prediccion.csv'")
df_train, df_test = dividir_train_test(df)

def crear_secuencias(datos, n_pasos):
    X, y = [], []
    for i in range(len(datos) - n_pasos):
        X.append(datos[i:i + n_pasos])
        y.append(datos[i + n_pasos])
    return np.array(X), np.array(y)

In [ ]:
# Configuración de hiperparámetros
model_type = 'SimpleRNN'  # Cambia por 'SimpleRNN', 'LSTM' o 'GRU' para entrenar otros modelos
loss_function = 'mse'  # Puedes cambiar por 'mse' o 'mae'
n_pasos = 24
n_epochs = 40
batch_size = 32
unidades = 50

# Crear secuencias
X_train, y_train = crear_secuencias(df_train['value_scaled'].values, n_pasos)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Crear secuencias para Test
X_test, y_test = crear_secuencias(df_test['value_scaled'].values, n_pasos)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Definir el modelo
if model_type == 'SimpleRNN':
    rnn_layer = SimpleRNN(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'LSTM':
    rnn_layer = LSTM(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'GRU':
    rnn_layer = GRU(unidades, activation='tanh', input_shape=(n_pasos, 1))

model = Sequential([
    rnn_layer,
    Dense(1)
])

model.compile(optimizer='adam', loss=loss_function)

# Entrenamiento
history = model.fit(
    X_train, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),  # ⚙️ Esto es obligatorio para tener val_loss
    verbose=1
)

# Guardar el scaler
#model.save(f'{model_type}_model_{loss_function}.keras')

with open(f'scaler_{model_type}_{loss_function}.pkl', 'wb') as f:
    pickle.dump(scaler, f)
    
# Guardar el history
with open(f"{model_type}_history_{loss_function}.pkl", 'wb') as f:
    pickle.dump(history.history, f)

# Guardar el modelo en ONNX
# 1. Crear el input_signature
input_signature = [tf.TensorSpec([None, n_pasos, 1], tf.float32, name="input")]

# 2. Decorar la función (no obtener concrete_func)
@tf.function(input_signature=input_signature)
def model_func(x):
    return model(x)

# 3. Convertir a ONNX directamente desde la función decorada
onnx_model, _ = tf2onnx.convert.from_function(
    model_func,  # 👉 Aquí debes pasar la función decorada, no la concrete_func
    input_signature=input_signature,
    opset=13,
    output_path=f'{model_type}_model_{loss_function}.onnx'
)

print(f'✅ Modelo {model_type}_model_{loss_function}.onnx entrenado y guardado correctamente.')
print(f'Scaler {model_type} guardado.')
print(f"Historial de entrenamiento {model_type} guardado.")

In [ ]:
# Gráfico de pérdida
fig_loss = px.line(y=history.history['loss'], title='Pérdida durante el entrenamiento')
fig_loss.show()

In [ ]:
# Configuración de hiperparámetros
model_type = 'LSTM'  # Cambia por 'SimpleRNN', 'LSTM' o 'GRU' para entrenar otros modelos
loss_function = 'mse'  # Puedes cambiar por 'mse' o 'mae'
n_pasos = 24
n_epochs = 40
batch_size = 32
unidades = 50

# Crear secuencias
X_train, y_train = crear_secuencias(df_train['value_scaled'].values, n_pasos)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Crear secuencias para Test
X_test, y_test = crear_secuencias(df_test['value_scaled'].values, n_pasos)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Definir el modelo
if model_type == 'SimpleRNN':
    rnn_layer = SimpleRNN(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'LSTM':
    rnn_layer = LSTM(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'GRU':
    rnn_layer = GRU(unidades, activation='tanh', input_shape=(n_pasos, 1))

model = Sequential([
    rnn_layer,
    Dense(1)
])

model.compile(optimizer='adam', loss=loss_function)

# Entrenamiento
history = model.fit(
    X_train, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),  # ⚙️ Esto es obligatorio para tener val_loss
    verbose=1
)

# Guardar el scaler
#model.save(f'{model_type}_model_{loss_function}.keras')

with open(f'scaler_{model_type}_{loss_function}.pkl', 'wb') as f:
    pickle.dump(scaler, f)
    
# Guardar el history
with open(f"{model_type}_history_{loss_function}.pkl", 'wb') as f:
    pickle.dump(history.history, f)

# Guardar el modelo en ONNX
# 1. Crear el input_signature
input_signature = [tf.TensorSpec([None, n_pasos, 1], tf.float32, name="input")]

# 2. Decorar la función (no obtener concrete_func)
@tf.function(input_signature=input_signature)
def model_func(x):
    return model(x)

# 3. Convertir a ONNX directamente desde la función decorada
onnx_model, _ = tf2onnx.convert.from_function(
    model_func,  # 👉 Aquí debes pasar la función decorada, no la concrete_func
    input_signature=input_signature,
    opset=13,
    output_path=f'{model_type}_model_{loss_function}.onnx'
)

print(f'✅ Modelo {model_type}_model_{loss_function}.onnx entrenado y guardado correctamente.')
print(f'Scaler {model_type} guardado.')
print(f"Historial de entrenamiento {model_type} guardado.")

In [ ]:
px.line(y=history.history['loss'], title='Pérdida durante el entrenamiento').show()

In [ ]:
# Configuración de hiperparámetros
model_type = 'GRU'  # Cambia por 'SimpleRNN', 'LSTM' o 'GRU' para entrenar otros modelos
loss_function = 'mse'  # Puedes cambiar por 'mse' o 'mae'
n_pasos = 24
n_epochs = 40
batch_size = 32
unidades = 50

# Crear secuencias
X_train, y_train = crear_secuencias(df_train['value_scaled'].values, n_pasos)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Crear secuencias para Test
X_test, y_test = crear_secuencias(df_test['value_scaled'].values, n_pasos)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Definir el modelo
if model_type == 'SimpleRNN':
    rnn_layer = SimpleRNN(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'LSTM':
    rnn_layer = LSTM(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'GRU':
    rnn_layer = GRU(unidades, activation='tanh', input_shape=(n_pasos, 1))

model = Sequential([
    rnn_layer,
    Dense(1)
])

model.compile(optimizer='adam', loss=loss_function)

# Entrenamiento
history = model.fit(
    X_train, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),  # ⚙️ Esto es obligatorio para tener val_loss
    verbose=1
)

# Guardar el scaler
#model.save(f'{model_type}_model_{loss_function}.keras')

with open(f'scaler_{model_type}_{loss_function}.pkl', 'wb') as f:
    pickle.dump(scaler, f)
    
# Guardar el history
with open(f"{model_type}_history_{loss_function}.pkl", 'wb') as f:
    pickle.dump(history.history, f)

# Guardar el modelo en ONNX
# 1. Crear el input_signature
input_signature = [tf.TensorSpec([None, n_pasos, 1], tf.float32, name="input")]

# 2. Decorar la función (no obtener concrete_func)
@tf.function(input_signature=input_signature)
def model_func(x):
    return model(x)

# 3. Convertir a ONNX directamente desde la función decorada
onnx_model, _ = tf2onnx.convert.from_function(
    model_func,  # 👉 Aquí debes pasar la función decorada, no la concrete_func
    input_signature=input_signature,
    opset=13,
    output_path=f'{model_type}_model_{loss_function}.onnx'
)

print(f'✅ Modelo {model_type}_model_{loss_function}.onnx entrenado y guardado correctamente.')
print(f'Scaler {model_type} guardado.')
print(f"Historial de entrenamiento {model_type} guardado.")

In [ ]:
px.line(y=history.history['loss'], title='Pérdida durante el entrenamiento').show()

In [ ]:
# Configuración de hiperparámetros
model_type = 'SimpleRNN'  # Cambia por 'SimpleRNN', 'LSTM' o 'GRU' para entrenar otros modelos
loss_function = 'mae'  # Puedes cambiar por 'mse' o 'mae'
n_pasos = 24
n_epochs = 50
batch_size = 32
unidades = 50

# Crear secuencias
X_train, y_train = crear_secuencias(df_train['value_scaled'].values, n_pasos)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Crear secuencias para Test
X_test, y_test = crear_secuencias(df_test['value_scaled'].values, n_pasos)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Definir el modelo
if model_type == 'SimpleRNN':
    rnn_layer = SimpleRNN(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'LSTM':
    rnn_layer = LSTM(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'GRU':
    rnn_layer = GRU(unidades, activation='tanh', input_shape=(n_pasos, 1))

model = Sequential([
    rnn_layer,
    Dense(1)
])

model.compile(optimizer='adam', loss=loss_function)

# Entrenamiento
history = model.fit(
    X_train, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),  # ⚙️ Esto es obligatorio para tener val_loss
    verbose=1
)

# Guardar el scaler
#model.save(f'{model_type}_model_{loss_function}.keras')

with open(f'scaler_{model_type}_{loss_function}.pkl', 'wb') as f:
    pickle.dump(scaler, f)
    
# Guardar el history
with open(f"{model_type}_history_{loss_function}.pkl", 'wb') as f:
    pickle.dump(history.history, f)

# Guardar el modelo en ONNX
# 1. Crear el input_signature
input_signature = [tf.TensorSpec([None, n_pasos, 1], tf.float32, name="input")]

# 2. Decorar la función (no obtener concrete_func)
@tf.function(input_signature=input_signature)
def model_func(x):
    return model(x)

# 3. Convertir a ONNX directamente desde la función decorada
onnx_model, _ = tf2onnx.convert.from_function(
    model_func,  # 👉 Aquí debes pasar la función decorada, no la concrete_func
    input_signature=input_signature,
    opset=13,
    output_path=f'{model_type}_model_{loss_function}.onnx'
)

print(f'✅ Modelo {model_type}_model_{loss_function}.onnx entrenado y guardado correctamente.')
print(f'Scaler {model_type} guardado.')
print(f"Historial de entrenamiento {model_type} guardado.")

In [ ]:
px.line(y=history.history['loss'], title='Pérdida durante el entrenamiento').show()

In [ ]:
# Configuración de hiperparámetros
model_type = 'LSTM'  # Cambia por 'SimpleRNN', 'LSTM' o 'GRU' para entrenar otros modelos
loss_function = 'mae'  # Puedes cambiar por 'mse' o 'mae'
n_pasos = 24
n_epochs = 50
batch_size = 32
unidades = 50

# Crear secuencias
X_train, y_train = crear_secuencias(df_train['value_scaled'].values, n_pasos)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Crear secuencias para Test
X_test, y_test = crear_secuencias(df_test['value_scaled'].values, n_pasos)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Definir el modelo
if model_type == 'SimpleRNN':
    rnn_layer = SimpleRNN(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'LSTM':
    rnn_layer = LSTM(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'GRU':
    rnn_layer = GRU(unidades, activation='tanh', input_shape=(n_pasos, 1))

model = Sequential([
    rnn_layer,
    Dense(1)
])

model.compile(optimizer='adam', loss=loss_function)

# Entrenamiento
history = model.fit(
    X_train, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),  # ⚙️ Esto es obligatorio para tener val_loss
    verbose=1
)

# Guardar el scaler
#model.save(f'{model_type}_model_{loss_function}.keras')

with open(f'scaler_{model_type}_{loss_function}.pkl', 'wb') as f:
    pickle.dump(scaler, f)
    
# Guardar el history
with open(f"{model_type}_history_{loss_function}.pkl", 'wb') as f:
    pickle.dump(history.history, f)

# Guardar el modelo en ONNX
# 1. Crear el input_signature
input_signature = [tf.TensorSpec([None, n_pasos, 1], tf.float32, name="input")]

# 2. Decorar la función (no obtener concrete_func)
@tf.function(input_signature=input_signature)
def model_func(x):
    return model(x)

# 3. Convertir a ONNX directamente desde la función decorada
onnx_model, _ = tf2onnx.convert.from_function(
    model_func,  # 👉 Aquí debes pasar la función decorada, no la concrete_func
    input_signature=input_signature,
    opset=13,
    output_path=f'{model_type}_model_{loss_function}.onnx'
)

print(f'✅ Modelo {model_type}_model_{loss_function}.onnx entrenado y guardado correctamente.')
print(f'Scaler {model_type} guardado.')
print(f"Historial de entrenamiento {model_type} guardado.")

In [ ]:
px.line(y=history.history['loss'], title='Pérdida durante el entrenamiento').show()

In [ ]:
# Configuración de hiperparámetros
model_type = 'GRU'  # Cambia por 'SimpleRNN', 'LSTM' o 'GRU' para entrenar otros modelos
loss_function = 'mae'  # Puedes cambiar por 'mse' o 'mae'
n_pasos = 24
n_epochs = 50
batch_size = 32
unidades = 50

# Crear secuencias
X_train, y_train = crear_secuencias(df_train['value_scaled'].values, n_pasos)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Crear secuencias para Test
X_test, y_test = crear_secuencias(df_test['value_scaled'].values, n_pasos)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Definir el modelo
if model_type == 'SimpleRNN':
    rnn_layer = SimpleRNN(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'LSTM':
    rnn_layer = LSTM(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'GRU':
    rnn_layer = GRU(unidades, activation='tanh', input_shape=(n_pasos, 1))

model = Sequential([
    rnn_layer,
    Dense(1)
])

model.compile(optimizer='adam', loss=loss_function)

# Entrenamiento
history = model.fit(
    X_train, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),  # ⚙️ Esto es obligatorio para tener val_loss
    verbose=1
)

# Guardar el scaler
#model.save(f'{model_type}_model_{loss_function}.keras')

with open(f'scaler_{model_type}_{loss_function}.pkl', 'wb') as f:
    pickle.dump(scaler, f)
    
# Guardar el history
with open(f"{model_type}_history_{loss_function}.pkl", 'wb') as f:
    pickle.dump(history.history, f)

# Guardar el modelo en ONNX
# 1. Crear el input_signature
input_signature = [tf.TensorSpec([None, n_pasos, 1], tf.float32, name="input")]

# 2. Decorar la función (no obtener concrete_func)
@tf.function(input_signature=input_signature)
def model_func(x):
    return model(x)

# 3. Convertir a ONNX directamente desde la función decorada
onnx_model, _ = tf2onnx.convert.from_function(
    model_func,  # 👉 Aquí debes pasar la función decorada, no la concrete_func
    input_signature=input_signature,
    opset=13,
    output_path=f'{model_type}_model_{loss_function}.onnx'
)

print(f'✅ Modelo {model_type}_model_{loss_function}.onnx entrenado y guardado correctamente.')
print(f'Scaler {model_type} guardado.')
print(f"Historial de entrenamiento {model_type} guardado.")

In [ ]:
px.line(y=history.history['loss'], title='Pérdida durante el entrenamiento').show()

In [ ]:
# Cargar datos para Prophet
df_prophet = get_demanda_data()
df_prophet['datetime'] = df_prophet['datetime'].dt.tz_localize(None)
df_prophet = df_prophet.rename(columns={"datetime": "ds", "value": "y"})

In [ ]:
# Guardar en csv 
df_prophet.to_csv('datos_prediccion_prophet.csv', index=False)

In [ ]:
# Definir granularidades
granularidades = {
    'D': 'diaria',
    'W': 'semanal',
    'ME': 'mensual',
    'QE': 'trimestral',
    '2Q': 'semestral',
    'YE': 'anual'
}

# Horizontes de predicción
horizontes = [10, 50, 100]

for freq, nombre in granularidades.items():
    print(f'Entrenando modelo para granularidad: {nombre}')
    
    # Reagrupar datos según granularidad
    df_freq = df_prophet.copy()
    df_freq = df_freq.groupby(pd.Grouper(key='ds', freq=freq)).sum().reset_index()

    model_prophet = Prophet()
    model_prophet.fit(df_freq)

    # Guardar modelo (por si quieres seguir usando más adelante)
    joblib.dump(model_prophet, f'prophet_model_{nombre}.joblib')

    # Precalcular predicciones para diferentes horizontes
    for n_pred in horizontes:
        print(f'Generando predicción para {n_pred} pasos')

        future = model_prophet.make_future_dataframe(periods=n_pred, freq=freq)
        forecast = model_prophet.predict(future)

        # Guardar predicción
        forecast.to_csv(f'forecast_{nombre}_{n_pred}.csv', index=False)

    print(f'✅ Modelo Prophet {nombre} y predicciones guardados correctamente.')

print('🎉 Todos los modelos y predicciones entrenados y guardados.')